In [41]:
# Dependencies
import numpy as np 
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import gc
import time
from tqdm import tqdm, tqdm_notebook; tqdm.pandas() # Progress bar
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
import params as model_params
# Machine Learning
import tensorflow as tf
from keras import backend as K
import tensorflow_io as tfio

#from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Model
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import features as features_lib
import params as model_params
import warnings,sys,os
warnings.filterwarnings("ignore")

if sys.platform == 'darwin':
    user_env = os.environ['PWD']
elif sys.platform == 'win32':
    user_env = os.environ['USERPROFILE']
   # model_google = models.KeyedVectors.load_word2vec_format(
   # 'C:\\Users\P70070004\Desktop\Sound_Ontology_Research\Data\GoogleNews-vectors-negative300.bin.gz', binary=True)
elif sys.platform == 'linux':
    user_env = '/workspace/notebooks/'
    
import nltk
nltk.download('punkt')

mode = 0
if user_env=='/workspace/notebooks/':
    project_path = os.path.join(user_env,'Natural-Sound-Analysis','')
    data_path = os.path.join(user_env,'Data','')
    if mode !=2:
        print('loading word2vec model...')
    dataframe_path = '/workspace/notebooks/Natural-Sound-Analysis/dataframes/'

else:
    project_path = os.path.join(user_env,'Desktop','NaturalSoundsAnalysis','Natural-Sound-Analysis','')
    data_path = os.path.join(user_env,'Desktop','NaturalSoundsAnalysis','Data','')
BATCH_SIZE = 16
double_gpu = 0
if double_gpu:
    GPUS = ["GPU:0", "GPU:1"]
    strategy = tf.distribute.MirroredStrategy( GPUS )
    batch_size = BATCH_SIZE * strategy.num_replicas_in_sync
    print('Number of devices: %d' % strategy.num_replicas_in_sync) 
else:
    GPUS = ['GPU:0']
    batch_size = BATCH_SIZE 
params = model_params.Params()
AUTOTUNE = tf.data.AUTOTUNE

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P70070004\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
def load_wav_16k_mono(filename, idx_frame = None):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
        file_contents,
        desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)


    target_length = 16000 * 5
    one_second = 0  # Length of one second of audio at 16kHz for the silence in the beginning
    
    current_length = tf.shape(wav)[0]

    if current_length < target_length - one_second:
        initial_padding = one_second
        sound_portion_length = current_length
        
        full_repeats = (target_length - one_second) // sound_portion_length
        remainder = (target_length - one_second) % sound_portion_length
        
        # Repeat the waveform and concatenate the remainder
        repeated_wav = tf.tile(wav, [full_repeats])
        remainder_wav = wav[:remainder]
        wav = tf.concat([repeated_wav, remainder_wav], axis=0)

        wav = tf.concat([tf.zeros(one_second, dtype=wav.dtype), wav], axis=0)
    else:
        if idx_frame is None:
            wav = tf.concat([tf.zeros(one_second, dtype=wav.dtype), wav[:16000*5]], axis=0)
        else:
            start_index = target_length * (idx_frame - 1)
            end_index = start_index + target_length
            wav = tf.concat([tf.zeros(one_second, dtype=wav.dtype), wav[start_index:end_index]], axis=0)
                       
            if tf.shape(wav)[0] < target_length + one_second:
                padding = target_length + one_second - tf.shape(wav)[0]
                wav = tf.concat([wav, tf.zeros(padding, dtype=wav.dtype)], axis=0)
    return wav
def get_spectrogram_and_label_id(audio, params, idx_silence):
    # Convert waveform to log-mel spectrogram patches
    features = features_lib.waveform_to_log_mel_spectrogram_patches(audio, params)

    return features
def buffer_and_select_states(features, buffer_size):
    # Initialize an empty tensor to accumulate buffers
    output_buffers = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    #print(features.shape[-1])
    # Initialize the buffer with zeros
    initial_buffer = tf.zeros((buffer_size, features.shape[-1]))

    # Define a function to slide the window and update the buffer
    def body(i, buffer, ta):
        # Update the buffer by sliding the window
        new_buffer = tf.concat([buffer[1:], [features[i]]], axis=0)
        # Write the updated buffer to TensorArray
        ta = ta.write(i, new_buffer)
        return i + 1, new_buffer, ta

    # Run the loop over all frames
    _, _, final_ta = tf.while_loop(
        cond=lambda i, *_: i < tf.shape(features)[0],
        body=body,
        loop_vars=(0, initial_buffer, output_buffers),
        parallel_iterations=1  # Ensure correct ordering
    )
    
    # Stack all buffered states
    stacked_buffers = final_ta.stack()
    
    return stacked_buffers
def load_and_preprocess_combined(file_path, params, silence_index, idx_frame_train=None):
    if idx_frame_train is None:
        audio = load_wav_16k_mono(file_path)
    else:
        audio = load_wav_16k_mono(file_path, idx_frame_train)

    spectrogram = get_spectrogram_and_label_id(audio, params, silence_index)
    #print(updated_labels)
    return spectrogram
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Dropout, MaxPooling2D,
                                     GRU, TimeDistributed, Softmax, Dense, Flatten,Permute, Masking, Bidirectional, Lambda)
from tensorflow.keras.models import Model
class FrequencyAttention(Layer):
    def __init__(self, **kwargs):
        super(FrequencyAttention, self).__init__(**kwargs)
        self.softmax = Softmax(axis=2)  # Applying softmax over the frequency dimension

    def build(self, input_shape):
        # This builds a weight vector for the frequency dimension
        self.attention_weights = self.add_weight(
            shape=(input_shape[2], 1),  # Frequency is the third dimension in input_shape
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        # Apply attention across the frequency dimension
        # inputs shape: (batch, time, frequency, channels)
        # Reduce across the channel to simplify the attention application
        reduced = tf.reduce_mean(inputs, axis=-1, keepdims=True)  # Shape: (batch, time, frequency, 1)
        # Compute attention scores using a simple dot product (not matrix multiplication)
        scores = tf.reduce_sum(reduced * self.attention_weights, axis=2, keepdims=True)  # Shape: (batch, time, 1, 1)
        scores = self.softmax(scores)  # Apply softmax to obtain attention scores

        # Apply attention weights
        attended = inputs * scores  # Broadcasting over the frequency and channel dimensions

        return attended

    def compute_output_shape(self, input_shape):
        return input_shape

def apply_attention_with_permute(x):
    # Assuming x has shape (batch, time, height, width, channels)
    # and you want to focus attention on the frequency (height or width depending on data)

    # Permute dimensions to focus on 'height' or 'width' for attention
    # For example, to focus on 'width', permute to bring 'width' as the last dimension before channels
    x = Permute((1, 3, 2, 4))(x)  # New shape: (batch, time, width, height, channels)
    x = TimeDistributed(FrequencyAttention())(x)
    x = Permute((1, 3, 2, 4))(x)  # Revert back to original: (batch, time, height, width, channels)
    
    return x

class GRUAttention(Layer):
    def __init__(self, **kwargs):
        super(GRUAttention, self).__init__(**kwargs)
        self.softmax = Softmax(axis=-1)

    def build(self, input_shape):
        # Assume attention is calculated across the feature dimension
        self.attention_weights = self.add_weight(
            shape=(input_shape[2], 1),  # input_shape[2] is the feature dimension of the GRU output
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        # Applying attention across the feature dimension
        x = tf.linalg.matmul(inputs, self.attention_weights)  # [batch_size, timesteps, 1]
        x = self.softmax(x)
        x = tf.linalg.matmul(inputs, x, transpose_a=True)  # Weighted sum of features across time
        return x  # [batch_size, feature_dim, 1]

    def compute_output_shape(self, input_shape):
        # Returns the same feature dimension with the number of timesteps reduced to 1
        return (input_shape[0], input_shape[2], 1)

def get_buffered_inputs(wav, params, buffer_sizes):
    wav = tf.convert_to_tensor(wav, dtype=tf.float32)
    spec = get_spectrogram_and_label_id(wav, params, silence_index)
    expected_frames = 498
    if spec.shape[0] > expected_frames:
        spec = spec[:expected_frames, :]
    elif spec.shape[0] < expected_frames:
        pad_len = expected_frames - spec.shape[0]
        spec = tf.pad(spec, [[0, pad_len], [0, 0]])
    return [
        np.expand_dims(buffer_and_select_states(spec, buffer_sizes[0]), axis=0),
        np.expand_dims(buffer_and_select_states(spec, buffer_sizes[1]), axis=0),
        np.expand_dims(buffer_and_select_states(spec, buffer_sizes[2]), axis=0),
    ]
    
#base_path = '/workspace/notebooks/Data/AUD_LSTMCNN_weights/final_model_FromGridSearch/'
base_path = "C:/Users/P70070004/Desktop/NaturalSoundsAnalysis/Data/"
fine_tune_path = base_path + 'finetunedESC50/'
dataframe_path = "C:/Users/P70070004/Desktop/NaturalSoundsAnalysis/Natural-Sound-Analysis/dataframes/"

In [43]:
from tensorflow.keras.models import load_model, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import backend as K

predictions_dict = {}

# Define model list and corresponding buffer size sets
model_buffer_map = {
    'CRNN_optimised_ESC_2025_248.h5': [2, 4, 8],
    'CRNN_optimised_ESC_2025_52040.h5': [5, 20, 40]
}

df_eval_sel = pd.read_csv(dataframe_path + 'ESC-50/esc50.csv')
train_df = df_eval_sel[df_eval_sel['fold'].isin([1, 2, 3, 4])]
test_df = df_eval_sel[df_eval_sel['fold'] == 5]

files_train = [data_path + 'ESC-50/audio/' + str(x) for x in train_df['filename']]
labels_train = list(train_df['category'])
files_test = [data_path + 'ESC-50/audio/' + str(x) for x in test_df['filename']]
labels_test = list(test_df['category'])

custom_objects = {
    'FrequencyAttention': FrequencyAttention,
    'GRUAttention': GRUAttention
}
# One-hot encoding labels
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(pd.DataFrame(labels_train + labels_test))
test_labels = encoder.transform(pd.DataFrame(labels_test))
print("Encoded classes:", encoder.categories_[0])

params = model_params.Params()
params.stft_window_seconds = 0.025

silence_index = 0
from sklearn.preprocessing import MultiLabelBinarizer


df_eval_sel = pd.read_csv(dataframe_path + 'ESC-50/esc50.csv')

# Selecting elements for train and test DataFrames
train_df = df_eval_sel[df_eval_sel['fold'].isin([1, 2, 3, 4])]
test_df = df_eval_sel[df_eval_sel['fold'] == 5]

files_train =list(train_df['filename'])
files_train = [data_path+'ESC-50/audio/' + str(x) for x in files_train]  
labels_train =list(train_df['category'])

files_test =list(test_df['filename'])
files_test = [data_path+'ESC-50/audio/' + str(x) for x in files_test]  
labels_test = list(test_df['category'])
all_labels = labels_train + labels_test

labels_list_train = list(all_labels)
#label_enc = LabelEncoder()
encoder = OneHotEncoder(sparse_output = False)
#y_int = label_enc.fit_transform(events)
#y_int = y_int.reshape(len(y_int), 1)
encoder.fit(pd.DataFrame(labels_list_train))
test_labels = encoder.transform(pd.DataFrame(labels_test))
print("Encoded classes:", encoder.categories_[0])

params = model_params.Params()
params.stft_window_seconds = 0.025

def load_and_preprocess_combined(file_path, params, silence_index, idx_frame_train=None):
    if idx_frame_train is None:
        audio = load_wav_16k_mono(file_path)
    else:
        audio = load_wav_16k_mono(file_path, idx_frame_train)
    
    spectrogram = get_spectrogram_and_label_id(audio, params, silence_index)
    #print(updated_labels)
    return spectrogram

silence_index = 0
test_dataset = tf.data.Dataset.from_tensor_slices(
    (files_test, None)).map(
        lambda x, z: load_and_preprocess_combined(x, params, silence_index, z)
    )


model_name, buffer_sizes = next(iter(model_buffer_map.items()))
#model_name, buffer_sizes = next(reversed(model_buffer_map.items()))

print(f"\nProcessing model: {model_name} with buffers: {buffer_sizes}")
# === Load the trained model ===
#model_name = 'CRNN_optimised_ESC_2025_248.h5'
model_path = fine_tune_path + '/' + model_name
model = load_model(model_path, custom_objects=custom_objects)
#model.summary()

Encoded classes: ['airplane' 'breathing' 'brushing_teeth' 'can_opening' 'car_horn' 'cat'
 'chainsaw' 'chirping_birds' 'church_bells' 'clapping' 'clock_alarm'
 'clock_tick' 'coughing' 'cow' 'crackling_fire' 'crickets' 'crow'
 'crying_baby' 'dog' 'door_wood_creaks' 'door_wood_knock'
 'drinking_sipping' 'engine' 'fireworks' 'footsteps' 'frog'
 'glass_breaking' 'hand_saw' 'helicopter' 'hen' 'insects'
 'keyboard_typing' 'laughing' 'mouse_click' 'pig' 'pouring_water' 'rain'
 'rooster' 'sea_waves' 'sheep' 'siren' 'sneezing' 'snoring' 'thunderstorm'
 'toilet_flush' 'train' 'vacuum_cleaner' 'washing_machine' 'water_drops'
 'wind']
Encoded classes: ['airplane' 'breathing' 'brushing_teeth' 'can_opening' 'car_horn' 'cat'
 'chainsaw' 'chirping_birds' 'church_bells' 'clapping' 'clock_alarm'
 'clock_tick' 'coughing' 'cow' 'crackling_fire' 'crickets' 'crow'
 'crying_baby' 'dog' 'door_wood_creaks' 'door_wood_knock'
 'drinking_sipping' 'engine' 'fireworks' 'footsteps' 'frog'
 'glass_breaking' 'hand_saw'

In [44]:
# === Choose the layer name you want to extract from ===
#target_layer_name = 'concatenate_streams'  # exact name

target_layer_name = 'dense_2' 
# === Create a model that outputs embeddings from that layer ===
embedding_model = Model(inputs=model.inputs,
                        outputs=model.get_layer(target_layer_name).output)

n_sounds = 400

for i, file in enumerate(files_test):
    wav = load_wav_16k_mono(file)
    inputs = get_buffered_inputs(wav, params, buffer_sizes=buffer_sizes)
    emb = embedding_model.predict(inputs)[0]  # shape: [T, D]
    
    # allocate shape dynamically on first iteration
    if i == 0:
        T, D = emb.shape
        embeddings_all_sounds = np.zeros((n_sounds, T, D))
        
    embeddings_all_sounds[i] = emb
    
    if i % 50 == 0:
        print(f"Processed {i}/{n_sounds} files")

print("✅ Done. Embeddings shape:", embeddings_all_sounds.shape)

1/1 [==============================] - 4s 4s/step
Processed 0/400 files
1/1 [==============================] - 0s 394ms/step
Processed 50/400 files
1/1 [==============================] - 0s 390ms/step
Processed 100/400 files
1/1 [==============================] - 0s 407ms/step
Processed 150/400 files
1/1 [==============================] - 0s 400ms/step
Processed 200/400 files
1/1 [==============================] - 0s 437ms/step
Processed 250/400 files
1/1 [==============================] - 0s 434ms/step
Processed 300/400 files
1/1 [==============================] - 0s 417ms/step
Processed 350/400 files
1/1 [==============================] - 0s 386ms/step
✅ Done. Embeddings shape: (400, 498, 50)


In [45]:
event_type_map = {
    # Quick Events
    'sneezing': 'Quick',
    'mouse_click': 'Quick',
    'glass_breaking': 'Quick',
    'can_opening': 'Quick',
    'door_wood_knock': 'Quick',
    'clock_alarm': 'Quick',
    'hand_saw': 'Quick',

    # Mid Events
    'dog': 'Mid',
    'cat': 'Mid',
    'cow': 'Mid',
    'crying_baby': 'Mid',
    'clapping': 'Mid',
    'coughing': 'Mid',
    'footsteps': 'Mid',
    'laughing': 'Mid',
    'brushing_teeth': 'Mid',
    'drinking_sipping': 'Mid',
    'pig': 'Mid',
    'rooster': 'Mid',
    'door_wood_creaks': 'Mid',
    'snoring': 'Mid',
    'frog': 'Mid',
    'hen': 'Mid',
    'crow': 'Mid',
    'fireworks': 'Mid',
    'airplane': 'Mid',
    'train': 'Mid',
    'car_horn': 'Mid',
    'engine': 'Mid',
    'church_bells': 'Mid',

    # Periodic Events
    'clock_tick': 'Periodic',
    'rain': 'Periodic',
    'sea_waves': 'Periodic',
    'crackling_fire': 'Periodic',
    'crickets': 'Periodic',
    'chirping_birds': 'Periodic',
    'water_drops': 'Periodic',
    'wind': 'Periodic',
    'pouring_water': 'Periodic',
    'thunderstorm': 'Periodic',
    'vacuum_cleaner': 'Periodic',
    'washing_machine': 'Periodic',
    'keyboard_typing': 'Periodic',
    'insects': 'Periodic',
    'chainsaw': 'Periodic',
    'helicopter': 'Periodic',
    'sheep': 'Periodic',
}

superclass_map = {
    # 🐾 Animals
    'dog': 'Animals',
    'rooster': 'Animals',
    'pig': 'Animals',
    'cow': 'Animals',
    'frog': 'Animals',
    'cat': 'Animals',
    'hen': 'Animals',
    'insects': 'Animals',
    'sheep': 'Animals',
    'crow': 'Animals',

    # 🌿 Natural sounds
    'rain': 'Natural sounds',
    'sea_waves': 'Natural sounds',
    'crackling_fire': 'Natural sounds',
    'crickets': 'Natural sounds',
    'chirping_birds': 'Natural sounds',
    'water_drops': 'Natural sounds',
    'wind': 'Natural sounds',
    'pouring_water': 'Natural sounds',
    'toilet_flush': 'Natural sounds',
    'thunderstorm': 'Natural sounds',

    # 🗣️ Human sounds
    'crying_baby': 'Human sounds',
    'sneezing': 'Human sounds',
    'clapping': 'Human sounds',
    'breathing': 'Human sounds',
    'coughing': 'Human sounds',
    'footsteps': 'Human sounds',
    'laughing': 'Human sounds',
    'brushing_teeth': 'Human sounds',
    'snoring': 'Human sounds',
    'drinking_sipping': 'Human sounds',

    # 🏠 Domestic sounds
    'door_wood_knock': 'Domestic sounds',
    'mouse_click': 'Domestic sounds',
    'keyboard_typing': 'Domestic sounds',
    'door_wood_creaks': 'Domestic sounds',
    'can_opening': 'Domestic sounds',
    'washing_machine': 'Domestic sounds',
    'vacuum_cleaner': 'Domestic sounds',
    'clock_alarm': 'Domestic sounds',
    'clock_tick': 'Domestic sounds',
    'glass_breaking': 'Domestic sounds',

    # 🚨 Urban sounds
    'helicopter': 'Urban sounds',
    'chainsaw': 'Urban sounds',
    'siren': 'Urban sounds',
    'car_horn': 'Urban sounds',
    'engine': 'Urban sounds',
    'train': 'Urban sounds',
    'church_bells': 'Urban sounds',
    'airplane': 'Urban sounds',
    'fireworks': 'Urban sounds',
    'hand_saw': 'Urban sounds'
}

test_df['event_type'] = test_df['category'].map(event_type_map)
test_df['superclass'] = test_df['category'].map(superclass_map)
test_df = test_df.reset_index()
test_df



,index,filename,fold,target,category,esc10,src_file,take,event_type,superclass
0,1600,5-103415-A-2.wav,5,2,pig,False,103415,A,Mid,Animals
1,1601,5-103416-A-2.wav,5,2,pig,False,103416,A,Mid,Animals
2,1602,5-103418-A-2.wav,5,2,pig,False,103418,A,Mid,Animals
3,1603,5-103420-A-2.wav,5,2,pig,False,103420,A,Mid,Animals
4,1604,5-103421-A-2.wav,5,2,pig,False,103421,A,Mid,Animals
...,...,...,...,...,...,...,...,...,...,...
395,1995,5-263831-B-6.wav,5,6,hen,False,263831,B,Mid,Animals
396,1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A,Periodic,Domestic sounds
397,1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A,Mid,Human sounds
398,1998,5-61635-A-8.wav,5,8,sheep,False,61635,A,Periodic,Animals


In [46]:
import numpy as np
from sklearn.metrics import pairwise_distances
from numpy.linalg import eigh, svd

def classical_mds(D, n_components=10):
    """
    Classical (metric) MDS: dato D (NxN) di distanze euclidee, 
    ritorna coordinate Nxk. Equivalente a PCA su matrice di Gram.
    """
    # D deve essere euclidea metrica (qui la calcoliamo noi)
    D2 = D ** 2
    n = D.shape[0]
    J = np.eye(n) - np.ones((n, n)) / n
    B = -0.5 * J @ D2 @ J  # double-centering

    # autovalori/vettori (prendiamo i top k)
    # eigh: simmetrica, ordina crescente → prendiamo la coda
    vals, vecs = eigh(B)
    idx = np.argsort(vals)[::-1]  # decrescente
    vals = vals[idx]
    vecs = vecs[:, idx]

    # tieni solo i positivi (numerica-mente robusto)
    pos = vals > 1e-12
    vals = vals[pos]
    vecs = vecs[:, pos]

    k = min(n_components, len(vals))
    L = np.diag(np.sqrt(vals[:k]))
    V = vecs[:, :k]
    X = V @ L   # Nxk
    return X

def procrustes_noscale(X, Y_ref):
    """
    Procrustes allineando X a Y_ref con sola traslazione+rotazione (no scaling).
    Restituisce X_allineata.
    """
    # centra entrambi
    Xc = X - X.mean(axis=0, keepdims=True)
    Yc = Y_ref - Y_ref.mean(axis=0, keepdims=True)

    # rotazione ortogonale via SVD (Orthogonal Procrustes, senza scala)
    # risolve min || Xc R - Yc || con R ortogonale
    U, _, Vt = svd(Xc.T @ Yc, full_matrices=False)
    R = U @ Vt  # rotazione

    X_aligned = Xc @ R + Y_ref.mean(axis=0, keepdims=True)  # reintroduci la traslazione target
    return X_aligned


In [47]:
def frame_distance_matrix(E_t, metric='euclid_norm'):
    """
    E_t: (N, D) embedding di tutti i suoni al frame t (tutti gli stream concatenati).
    metric: 'euclid_norm' (consigliata) oppure 'cosine'
    """
    if metric == 'euclid_norm':
        # L2-normalizza per riga → Euclidea su vettori unitari
        norms = np.linalg.norm(E_t, axis=1, keepdims=True) + 1e-12
        Z = E_t / norms
        D = pairwise_distances(Z, metric='euclidean')
    elif metric == 'cosine':
        D = pairwise_distances(E_t, metric='cosine')
    else:
        raise ValueError("metric deve essere 'euclid_norm' o 'cosine'")
    return D


In [48]:

from tqdm import tqdm
N, T, D = embeddings_all_sounds.shape

metric = 'cosine'      # euclid_norm oppure 'cosine'
k_mds  = 20                 # "max n dimensions per fare la procrustes" (10 è un buon compromesso)
ref_t  = 100                # frame di riferimento (peak F1)

# 1) MDS per ogni frame
coords_all = []
for t in tqdm(range(T)):
    E_t = embeddings_all_sounds[:, t, :]    # (N, D)
    D_t = frame_distance_matrix(E_t, metric=metric)
    X_t = classical_mds(D_t, n_components=k_mds)  # (N, k_mds)
    coords_all.append(X_t)
coords_all = np.stack(coords_all, axis=1)   # (N, T, k_mds)

# 2) Procrustes (no scaling) verso il frame di riferimento
X_ref = coords_all[:, ref_t, :]             # (N, k_mds)
coords_aligned = np.empty_like(coords_all)
for t in tqdm(range(T)):
    coords_aligned[:, t, :] = procrustes_noscale(coords_all[:, t, :], X_ref)

# 3) Prendi solo le prime 2 dimensioni per visualizzazione
coords2 = coords_aligned[:, :, :2]          # (N, T, 2)
print("coords2:", coords2.shape)


100%|██████████| 498/498 [00:00<00:00, 2246.98it/s]

coords2: (400, 498, 2)


In [49]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# === Color setup (Superclass colors) ===
superclasses = test_df['superclass'].values
uniq_super = np.unique(superclasses)
cmap = plt.cm.get_cmap('tab10', len(uniq_super))
color_map = {c:i for i,c in enumerate(uniq_super)}
colors = np.array([color_map[c] for c in superclasses])

# === Plot limits ===
xmin, xmax = coords2[:,:,0].min(), coords2[:,:,0].max()
ymin, ymax = coords2[:,:,1].min(), coords2[:,:,1].max()
padx = 0.05*(xmax-xmin+1e-8)
pady = 0.05*(ymax-ymin+1e-8)

# === Figure ===
fig, ax = plt.subplots(figsize=(15,15))
scat = ax.scatter(coords2[:,0,0], coords2[:,0,1],
                  c=colors, cmap=cmap, s=24, edgecolors='k', linewidths=0.3)
texts = []

ax.set_xlim(xmin-padx, xmax+padx)
ax.set_ylim(ymin-pady, ymax+pady)
ax.set_xlabel("MDS-1")
ax.set_ylabel("MDS-2")
ax.set_title(f"MDS (cosine) + Procrustes @ref {ref_t} — time 0")

# === Legend ===
handles = [plt.Line2D([0],[0], marker='o', color='w', label=c,
                      markerfacecolor=cmap(i), markersize=8)
           for i, c in enumerate(uniq_super)]
ax.legend(handles=handles, title="Superclass", loc="upper right", fontsize=7)

categories = test_df['category'].values
label_step = 4   # Show one label every 6 sounds

# === Frame sequence ===
frames_slow = list(range(0, 100, 2))      # first 100 frames slow
frames_fast = list(range(100, T, 8))      # rest faster
frames = frames_slow + frames_fast        # combine

# === Update function ===
def update(t):
    scat.set_offsets(coords2[:, t, :])
    global texts
    for txt in texts:
        txt.remove()
    texts = []

    for i, cat in enumerate(categories):
        if i % label_step == 0:
            txt = ax.text(coords2[i, t, 0], coords2[i, t, 1],
                          cat, fontsize=6, ha='right', va='center', alpha=0.8)
            texts.append(txt)

    ax.set_title(f"MDS (euclidean) + Procrustes @ref {ref_t} — frame {t}/{T-1}")
    return [scat] + texts

# === Variable speed: custom frame generator ===
class VariableSpeedAnimation(animation.FuncAnimation):
    def __init__(self, fig, func, frames, **kwargs):
        self.frame_intervals = []
        for f in frames:
            if f < 100:
                self.frame_intervals.append(1000)  # slow (~3x slower)
            else:
                self.frame_intervals.append(180)   # default speed
        self._counter = 0
        super().__init__(fig, func, frames=frames, interval=self.frame_intervals[0], **kwargs)
    
    def _step(self, *args):
        result = super()._step(*args)
        if self.event_source and self._counter < len(self.frame_intervals):
            self.event_source.interval = self.frame_intervals[self._counter]
        self._counter += 1
        return result

# === Animation ===
ani = VariableSpeedAnimation(fig, update, frames=frames, blit=False)

save_path = 'C:/Users/P70070004/Desktop/NaturalSoundsAnalysis/Causal-SemDNN/CRNN_oneSpectBuffer/Demo_Evaluate_networks/Results_Marseille25/'
# MP4 gives you play/pause control in any video viewer

import matplotlib.animation as animation

ani.save(
    save_path+ "mds_procrustes_concatenation_cosine_slowstart_20_40_80.avi",
    writer="ffmpeg",  
    fps=10,
    dpi=150
)
plt.close(fig)


In [50]:
# SET THIS to your probs array (N sounds, T frames, 50 classes)
probs_all_sounds = embeddings_all_sounds   # ← change if your var has a different name
N, T, C = probs_all_sounds.shape
class_names = np.unique(test_df['category'])
class_names

array(['airplane', 'breathing', 'brushing_teeth', 'can_opening',
       'car_horn', 'cat', 'chainsaw', 'chirping_birds', 'church_bells',
       'clapping', 'clock_alarm', 'clock_tick', 'coughing', 'cow',
       'crackling_fire', 'crickets', 'crow', 'crying_baby', 'dog',
       'door_wood_creaks', 'door_wood_knock', 'drinking_sipping',
       'engine', 'fireworks', 'footsteps', 'frog', 'glass_breaking',
       'hand_saw', 'helicopter', 'hen', 'insects', 'keyboard_typing',
       'laughing', 'mouse_click', 'pig', 'pouring_water', 'rain',
       'rooster', 'sea_waves', 'sheep', 'siren', 'sneezing', 'snoring',
       'thunderstorm', 'toilet_flush', 'train', 'vacuum_cleaner',
       'washing_machine', 'water_drops', 'wind'], dtype=object)

In [51]:
import numpy as np
from sklearn.metrics import pairwise_distances
from numpy.linalg import eigh, svd

def classical_mds(D, n_components=10):
    D2 = D**2
    n = D.shape[0]
    J = np.eye(n) - np.ones((n, n))/n
    B = -0.5 * J @ D2 @ J
    vals, vecs = eigh(B)
    idx = np.argsort(vals)[::-1]
    vals, vecs = vals[idx], vecs[:, idx]
    pos = vals > 1e-12
    vals, vecs = vals[pos], vecs[:, pos]
    k = min(n_components, len(vals))
    return vecs[:, :k] @ np.diag(np.sqrt(vals[:k]))

def procrustes_noscale(X, Y_ref):
    Xc = X - X.mean(axis=0, keepdims=True)
    Yc = Y_ref - Y_ref.mean(axis=0, keepdims=True)
    U, _, Vt = svd(Xc.T @ Yc, full_matrices=False)
    R = U @ Vt
    return Xc @ R + Y_ref.mean(axis=0, keepdims=True)
from tqdm import tqdm

metric = 'cosine'      # or 'euclidean'
k_mds  = 10
ref_t  = 177           # Bruno asked to align to peak F1 frame

I50 = np.eye(50, dtype=float)

coords_all = []  # list of (N+50, k_mds)
for t in tqdm(range(T), desc="MDS frames"):
    P_t = probs_all_sounds[:, t, :]           # (N, 50)
    X_t = np.vstack([P_t, I50])               # (N+50, 50)
    D_t = pairwise_distances(X_t, metric=metric)
    Y_t = classical_mds(D_t, n_components=k_mds)
    coords_all.append(Y_t)

coords_all = np.stack(coords_all, axis=1)     # (N+50, T, k_mds)

# Procrustes (no scaling) to ref frame
Y_ref = coords_all[:, ref_t, :]               # (N+50, k_mds)
coords_aligned = np.empty_like(coords_all)
for t in tqdm(range(T), desc="Procrustes align"):
    coords_aligned[:, t, :] = procrustes_noscale(coords_all[:, t, :], Y_ref)

# Take first 2 dims for viz
coords2 = coords_aligned[:, :, :2]            # (N+50, T, 2)
sound_idx = np.arange(N)                      # 0..N-1
label_idx = np.arange(N, N+50)                # N..N+49


Procrustes align: 100%|██████████| 498/498 [00:00<00:00, 2871.05it/s]


In [52]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# --- CONFIG ---
num_sound_labels = 60        # how many of the 400 sounds to label (tune this)
sound_label_strategy = "stratified"  # "every_k", "random", or "stratified"
every_k = 7                  # if strategy == "every_k", label every k-th sound
sound_label_fontsize = 6
class_label_fontsize = 7

# Colors for sounds (by superclass)
superclasses = test_df['superclass'].values     # shape (N,)
uniq_super = np.unique(superclasses)
cmap = plt.cm.get_cmap('tab10', len(uniq_super))
super_to_i = {c:i for i,c in enumerate(uniq_super)}
sound_colors = np.array([super_to_i[c] for c in superclasses])

# Select which sounds to label (indices in 0..N-1)
rng = np.random.RandomState(42)
if sound_label_strategy == "every_k":
    sound_label_indices = np.arange(0, N, every_k)
elif sound_label_strategy == "random":
    sound_label_indices = rng.choice(np.arange(N), size=min(num_sound_labels, N), replace=False)
elif sound_label_strategy == "stratified":
    # roughly balance across superclasses
    per_class = max(1, num_sound_labels // max(1, len(uniq_super)))
    idxs = []
    for sup in uniq_super:
        pool = np.where(superclasses == sup)[0]
        take = min(per_class, len(pool))
        if take > 0:
            idxs.append(rng.choice(pool, size=take, replace=False))
    sound_label_indices = np.concatenate(idxs) if len(idxs) else np.array([], dtype=int)
else:
    sound_label_indices = np.array([], dtype=int)
sound_label_indices = np.sort(np.unique(sound_label_indices))

# Category names for the 400 sounds and 50 classes
sound_categories = test_df['category'].values   # length N
# IMPORTANT: class_names must match the model output order 0..49
# Provide this list from your label encoder/order:
# class_names = your_list_of_50_class_names

# Plot bounds
xmin, xmax = coords2[:,:,0].min(), coords2[:,:,0].max()
ymin, ymax = coords2[:,:,1].min(), coords2[:,:,1].max()
padx = 0.05*(xmax-xmin+1e-8); pady = 0.05*(ymax-ymin+1e-8)

fig, ax = plt.subplots(figsize=(10,8))
sc_sounds = ax.scatter(coords2[sound_idx, 0, 0], coords2[sound_idx, 0, 1],
                       c=sound_colors, cmap=cmap, s=24, edgecolors='k', linewidths=0.3, label="Sounds")
sc_labels = ax.scatter(coords2[label_idx, 0, 0], coords2[label_idx, 0, 1],
                       c='none', edgecolors='black', s=70, marker='s', linewidths=1.0, label='True labels')

class_texts = []
sound_texts = []

ax.set_xlim(xmin-padx, xmax+padx); ax.set_ylim(ymin-pady, ymax+pady)
ax.set_xlabel("MDS-1"); ax.set_ylabel("MDS-2")
ax.set_title(f"Output-space MDS + Procrustes @ref {ref_t} (cosine) — time 0")

# Legend: superclasses + true label marker + sound text marker
handles = [plt.Line2D([0],[0], marker='o', color='w',
                      label=c, markerfacecolor=cmap(i), markersize=8)
           for i,c in enumerate(uniq_super)]
handles.append(plt.Line2D([0],[0], marker='s', color='k', mfc='none',
                          label='True label (class anchor)', markersize=8))
handles.append(plt.Line2D([0],[0], marker=None, color='black',
                          label='Sound label (subset)', linewidth=0))
ax.legend(handles=handles, title="Legend", loc="upper right", fontsize=7)

def draw_class_names(t):
    # draw labels for the 50 class squares
    for txt in class_texts: txt.remove()
    class_texts.clear()
    for j in range(50):
        x, y = coords2[N + j, t, 0], coords2[N + j, t, 1]
        txt = ax.text(x, y, class_names[j], fontsize=class_label_fontsize,
                      ha='left', va='center', alpha=0.95, color='black')
        class_texts.append(txt)

def draw_sound_names(t):
    # draw labels for a subset of sounds
    for txt in sound_texts: txt.remove()
    sound_texts.clear()
    for i in sound_label_indices:
        x, y = coords2[i, t, 0], coords2[i, t, 1]
        # color the text by superclass, or use a fixed contrasting color if you prefer
        txt_color = cmap(sound_colors[i] / max(1, len(uniq_super)-1))
        txt = ax.text(x, y, sound_categories[i], fontsize=sound_label_fontsize,
                      ha='right', va='center', alpha=0.85, color=txt_color)
        sound_texts.append(txt)

def update(t):
    sc_sounds.set_offsets(coords2[sound_idx, t, :])
    sc_labels.set_offsets(coords2[label_idx, t, :])
    draw_class_names(t)
    draw_sound_names(t)
    ax.set_title(f"Output-space MDS + Procrustes @ref {ref_t} (cosine) — time {t}/{T-1}")
    return [sc_sounds, sc_labels] + class_texts + sound_texts

# Frame cadence: faster after 100
frames = list(range(0, min(100, T), 2)) + list(range(100, T, 8))
ani = animation.FuncAnimation(fig, update, frames=frames, interval=80, blit=False)

# Save MP4 so you can pause/scrub
ani.save(save_path + "mds_20_40_80_cosine_plus_true_labels_anchors.mp4",
         writer="ffmpeg", fps=10, dpi=150)
plt.close(fig)


This is for concatenation layer analysis one-by-one

In [10]:
#embeddings_all_sounds_concatenate_248 = embeddings_all_sounds
#embeddings_all_sounds_concatenate_52040 = embeddings_all_sounds

from tensorflow.keras.models import Model
import numpy as np

def extract_embeddings_for_layer(model,
                                 target_layer_name,
                                 files_test,
                                 params,
                                 buffer_sizes,
                                 load_wav_fn=load_wav_16k_mono,
                                 get_buffered_inputs_fn=get_buffered_inputs,
                                 print_every=50):
    """
    Extract (N, T, D) embeddings from `target_layer_name` using your existing
    preprocessors (`load_wav_16k_mono` + `get_buffered_inputs`).
    Returns: embeddings_all_sounds (N, T, D)
    """
    # submodel for the chosen layer (exact name)
    embedding_model = Model(inputs=model.inputs,
                            outputs=model.get_layer(target_layer_name).output)

    n_sounds = len(files_test)
    embeddings_all_sounds = None
    T = D = None

    for i, file in enumerate(files_test):
        wav = load_wav_fn(file)
        inputs = get_buffered_inputs_fn(wav, params, buffer_sizes=buffer_sizes)
        emb = embedding_model.predict(inputs, verbose=0)[0]  # (T, D)

        if embeddings_all_sounds is None:
            T, D = emb.shape
            embeddings_all_sounds = np.zeros((n_sounds, T, D), dtype=emb.dtype)

        # sanity: all files should yield same (T, D)
        if emb.shape != (T, D):
            raise ValueError(f"Inconsistent shape at {file}: got {emb.shape}, expected {(T, D)}")

        embeddings_all_sounds[i] = emb

        if print_every and (i % print_every == 0):
            print(f"Processed {i}/{n_sounds} files")

    print(f"✅ Done. Embeddings shape: {embeddings_all_sounds.shape} (N={n_sounds}, T={T}, D={D})")
    return embeddings_all_sounds

from tqdm import tqdm
import numpy as np

def compute_mds_procrustes_series(embeddings_all_sounds,
                                  metric='cosine',
                                  k_mds=20,
                                  ref_t=100,
                                  use_tqdm=True):
    """
    embeddings_all_sounds: (N, T, D)
    metric: 'cosine' or 'euclid_norm' (as in frame_distance_matrix)
    k_mds: number of dims for MDS solution used for Procrustes (plot will use first 2)
    ref_t: reference frame index for Procrustes alignment (no scaling)
    returns: coords2 -> (N, T, 2)
    """
    N, T, D = embeddings_all_sounds.shape

    it = range(T)
    if use_tqdm:
        it = tqdm(it, desc="MDS per frame")

    # 1) MDS for each frame
    coords_all = []
    for t in it:
        E_t = embeddings_all_sounds[:, t, :]     # (N, D)
        D_t = frame_distance_matrix(E_t, metric=metric)
        X_t = classical_mds(D_t, n_components=k_mds)  # (N, k_mds)
        coords_all.append(X_t)
    coords_all = np.stack(coords_all, axis=1)   # (N, T, k_mds)

    # 2) Procrustes (no scaling) towards ref_t
    X_ref = coords_all[:, ref_t, :]             # (N, k_mds)
    coords_aligned = np.empty_like(coords_all)

    it2 = range(T)
    if use_tqdm:
        it2 = tqdm(it2, desc=f"Procrustes → ref {ref_t}")

    for t in it2:
        coords_aligned[:, t, :] = procrustes_noscale(coords_all[:, t, :], X_ref)

    # 3) First 2 dims for visualization
    coords2 = coords_aligned[:, :, :2]          # (N, T, 2)
    return coords2

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def animate_coords2(coords2,
                    test_df,
                    ref_t,
                    outpath="mds_procrustes.gif",
                    title_prefix="MDS (cosine) + Procrustes",
                    label_step=4,
                    figsize=(15, 15),
                    slow_until=100,     # frames < slow_until play slow
                    slow_step=2,        # stride in slow part
                    fast_step=8,        # stride after slow part
                    slow_interval=1000, # ms between frames for slow part
                    fast_interval=180   # ms between frames for fast part
                   ):
    """
    coords2: (N, T, 2)
    test_df: must have columns 'superclass' and 'category' aligned to N
    Saves GIF and returns the path.
    """
    N, T, _ = coords2.shape

    # Colors by superclass
    superclasses = test_df['superclass'].values
    uniq_super = np.unique(superclasses)
    cmap = plt.cm.get_cmap('tab10', len(uniq_super))
    sup2i = {c:i for i,c in enumerate(uniq_super)}
    colors = np.array([sup2i[c] for c in superclasses])

    # Fixed bounds
    xmin, xmax = coords2[:,:,0].min(), coords2[:,:,0].max()
    ymin, ymax = coords2[:,:,1].min(), coords2[:,:,1].max()
    padx = 0.05*(xmax-xmin+1e-8)
    pady = 0.05*(ymax-ymin+1e-8)

    # Figure
    fig, ax = plt.subplots(figsize=figsize)
    scat = ax.scatter(coords2[:,0,0], coords2[:,0,1],
                      c=colors, cmap=cmap, s=24, edgecolors='k', linewidths=0.3)
    texts = []

    ax.set_xlim(xmin-padx, xmax+padx)
    ax.set_ylim(ymin-pady, ymax+pady)
    ax.set_xlabel("MDS-1")
    ax.set_ylabel("MDS-2")
    ax.set_title(f"{title_prefix} @ref {ref_t} — time 0")

    # Legend
    handles = [plt.Line2D([0],[0], marker='o', color='w', label=c,
                          markerfacecolor=cmap(i), markersize=8)
               for i, c in enumerate(uniq_super)]
    ax.legend(handles=handles, title="Superclass", loc="upper right", fontsize=7)

    categories = test_df['category'].values

    def update(t):
        scat.set_offsets(coords2[:, t, :])
        nonlocal texts
        for txt in texts:
            txt.remove()
        texts = []
        for i, cat in enumerate(categories):
            if i % label_step == 0:
                txt = ax.text(coords2[i, t, 0], coords2[i, t, 1],
                              cat, fontsize=6, ha='right', va='center', alpha=0.8)
                texts.append(txt)
        ax.set_title(f"{title_prefix} @ref {ref_t} — frame {t}/{T-1}")
        return [scat] + texts

    # Frame schedule (slow then fast)
    frames_slow = list(range(0, min(slow_until, T), slow_step))
    frames_fast = list(range(min(slow_until, T), T, fast_step))
    frames = frames_slow + frames_fast

    # Variable-speed animation
    class VariableSpeedAnimation(animation.FuncAnimation):
        def __init__(self, fig, func, frames, **kwargs):
            self.frame_intervals = []
            for f in frames:
                if f < slow_until:
                    self.frame_intervals.append(slow_interval)
                else:
                    self.frame_intervals.append(fast_interval)
            self._counter = 0
            super().__init__(fig, func, frames=frames, interval=self.frame_intervals[0], **kwargs)
        def _step(self, *args):
            result = super()._step(*args)
            if self.event_source and self._counter < len(self.frame_intervals):
                self.event_source.interval = self.frame_intervals[self._counter]
            self._counter += 1
            return result

    ani = VariableSpeedAnimation(fig, update, frames=frames, blit=False)
    ani.save(outpath, writer="pillow", fps=10)
    plt.show()
    return outpath


In [11]:
# ==== 0) Split in 3 stream dal layer concatenato ====
def split_streams(emb_all):
    """
    emb_all: (N, T, Dtot) con Dtot multiplo di 3
    return: dict {"fast": (N,T,Ds), "mid": (N,T,Ds), "slow": (N,T,Ds)}
    """
    N, T, Dtot = emb_all.shape
    Ds = Dtot // 3
    return {
        "fast": emb_all[:, :, 0:Ds],
        "mid" : emb_all[:, :, Ds:2*Ds],
        "slow": emb_all[:, :, 2*Ds:3*Ds],
    }

# ==== 1) MDS+Procrustes (no scaling) per UNO stream ====
def mds_procrustes_single_stream(emb_stream, metric='cosine', k_mds=20, ref_t=100, use_tqdm=True):
    """
    emb_stream: (N, T, Ds)
    ritorna coords2: (N, T, 2) per quello stream
    """
    return compute_mds_procrustes_series(
        embeddings_all_sounds=emb_stream,
        metric=metric,
        k_mds=k_mds,
        ref_t=ref_t,
        use_tqdm=use_tqdm
    )

# ==== 2) MDS+Procrustes per TUTTI e 3 gli stream ====
def mds_all_streams(emb_all, metric='cosine', k_mds=20, ref_t=100, use_tqdm=True):
    """
    emb_all: (N, T, Dtot concatenato)
    return: dict {"fast": (N,T,2), "mid": (N,T,2), "slow": (N,T,2)}
    """
    streams = split_streams(emb_all)
    coords = {}
    for name, E in streams.items():
        print(f"[MDS] stream={name}  shape={E.shape}  metric={metric}  ref_t={ref_t}")
        coords[name] = mds_procrustes_single_stream(
            emb_stream=E, metric=metric, k_mds=k_mds, ref_t=ref_t, use_tqdm=use_tqdm
        )
    return coords

# ==== 3) GIF a 3 pannelli (MDS per stream), colori=superclass, etichette sparse=category ====
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def animate_three_streams_mds(coords_dict,
                              test_df,
                              ref_t,
                              outpath="mds_three_streams.gif",
                              title_prefix="MDS (cosine) + Procrustes",
                              label_every=8,
                              figsize=(24,8),
                              slow_until=100, slow_step=2,
                              fast_step=8,
                              slow_interval=800, fast_interval=120):
    """
    coords_dict: {"fast":(N,T,2), "mid":(N,T,2), "slow":(N,T,2)}
    test_df: deve avere colonne 'superclass' e 'category' allineate ai N suoni
    """
    order = ["fast","mid","slow"]
    Xs = [coords_dict[k] for k in order]
    N, T, _ = Xs[0].shape

    # colori per superclass
    superclasses = test_df['superclass'].values
    uniq_super = np.unique(superclasses)
    cmap = plt.cm.get_cmap('tab10', len(uniq_super))
    sup2i = {c:i for i,c in enumerate(uniq_super)}
    colors = np.array([sup2i[c] for c in superclasses])
    cats = test_df['category'].values

    # limiti comuni
    xmin = min(X[:,:,0].min() for X in Xs); xmax = max(X[:,:,0].max() for X in Xs)
    ymin = min(X[:,:,1].min() for X in Xs); ymax = max(X[:,:,1].max() for X in Xs)
    padx = 0.05*(xmax-xmin+1e-8); pady = 0.05*(ymax-ymin+1e-8)

    fig, axes = plt.subplots(1, 3, figsize=figsize, sharex=True, sharey=True)
    scatters, text_lists = [], []
    for ax, name, X in zip(axes, order, Xs):
        sc = ax.scatter(X[:,0,0], X[:,0,1], c=colors, cmap=cmap, s=30,
                        edgecolors='k', linewidths=0.3)
        scatters.append(sc)
        text_lists.append([])
        ax.set_xlim(xmin-padx, xmax+padx)
        ax.set_ylim(ymin-pady, ymax+pady)
        ax.set_title(f"{name} — t=0")
        ax.set_xlabel("MDS-1")
    axes[0].set_ylabel("MDS-2")

    # legenda
    handles = [plt.Line2D([0],[0], marker='o', color='w', label=c,
                          markerfacecolor=cmap(i), markersize=8)
               for i,c in enumerate(uniq_super)]
    axes[-1].legend(handles=handles, title="Superclass", fontsize=7, loc="upper right")

    # frames: lenti poi veloci
    frames_slow = list(range(0, min(slow_until, T), slow_step))
    frames_fast = list(range(min(slow_until, T), T, fast_step))
    frames = frames_slow + frames_fast

    def update(t):
        for ax, sc, texts, X, name in zip(axes, scatters, text_lists, Xs, order):
            sc.set_offsets(X[:, t, :])
            for txt in texts: txt.remove()
            texts.clear()
            for i, cat in enumerate(cats):
                if i % label_every == 0:
                    txt = ax.text(X[i, t, 0], X[i, t, 1],
                                  cat, fontsize=6, ha='right', va='center', alpha=0.8)
                    texts.append(txt)
            ax.set_title(f"{name} — frame {t}/{T-1}")
        fig.suptitle(f"{title_prefix} — ref {ref_t}", y=0.98)
        return scatters + [txt for texts in text_lists for txt in texts]

    class VariableSpeed(animation.FuncAnimation):
        def __init__(self, fig, func, frames, **kwargs):
            self.frame_intervals = [(slow_interval if f < slow_until else fast_interval) for f in frames]
            self._i = 0
            super().__init__(fig, func, frames=frames, interval=self.frame_intervals[0], **kwargs)
        def _step(self, *args):
            r = super()._step(*args)
            if self.event_source and self._i < len(self.frame_intervals):
                self.event_source.interval = self.frame_intervals[self._i]
            self._i += 1
            return r

    ani = VariableSpeed(fig, update, frames=frames, blit=False)
    ani.save(outpath, writer="pillow", fps=10)
    plt.show()
    return outpath


In [12]:
# embeddings_all_sounds_concatenate_248 = embeddings_all_sounds  # come da tua variabile
E248 = embeddings_all_sounds_concatenate_248  # (N, T, 768)

# 1) MDS+Procrustes per i tre stream
coords_streams_248 = mds_all_streams(
    emb_all=E248,
    metric='euclid_norm',   # oppure 'euclid_norm'
    k_mds=20,
    ref_t=100,         # oppure 177 se vuoi il peak F1 che usava Bruno
    use_tqdm=True
)

NameError: name 'embeddings_all_sounds_concatenate_248' is not defined

In [ ]:
# 2) GIF 3 pannelli
gif_248 = animate_three_streams_mds(
    coords_dict=coords_streams_248,
    test_df=test_df,
    ref_t=100,
    outpath="mds_three_streams_concat_euclid_248.gif",
    title_prefix="MDS (euclid) + Procrustes — stream-wise",
    label_every=6,
    figsize=(15,5),
    slow_until=100, slow_step=2,
    fast_step=8,
    slow_interval=600, fast_interval=120
)
print("Saved:", gif_248)

In [ ]:
E52040 = embeddings_all_sounds_concatenate_52040  # (N, T, 1536)

coords_streams_52040 = mds_all_streams(
    emb_all=E52040,
    metric='euclid_norm',
    k_mds=20,
    ref_t=100,
    use_tqdm=True
)

gif_52040 = animate_three_streams_mds(
    coords_dict=coords_streams_52040,
    test_df=test_df,
    ref_t=100,
    outpath="mds_three_streams_concatenate_euclidean_52040.gif",
    title_prefix="MDS (euclidean) + Procrustes — stream-wise",
    label_every=6,
    figsize=(15,5),
    slow_until=100, slow_step=2,
    fast_step=8,
    slow_interval=600, fast_interval=120
)
print("Saved:", gif_52040)
